## 01 | Channel Efficiency: Average Length of Stay (ALOS) by Distribution Source

**Business Question:** Which booking channels attract guests with the longest stays, and how does each channel contribute to the total room-nights generated?

In [ ]:
WITH channel_stay_data AS (
	SELECT
		COALESCE(cd.channel_name, 'Unmapped: ' || hb.distribution_channel) AS channel_name,
		(hb.stays_in_week_nights + hb.stays_in_weekend_nights) AS total_nights
	FROM hotel_bookings hb
	LEFT JOIN channel_descriptions cd
		ON cd.channel_code = hb.distribution_channel
	WHERE 
		hb.is_canceled = 0
		AND hb.distribution_channel <> 'Undefined'
)
SELECT
	csd.channel_name,
	COUNT(*) AS total_bookings,
	SUM(csd.total_nights) AS total_room_nights,
	ROUND(AVG(total_nights), 2) AS avg_length_of_stay
FROM channel_stay_data csd
GROUP BY csd.channel_name
ORDER BY avg_length_of_stay DESC;

### Visual Insight: Distribution Channel Efficiency (ALOS)

This visualization analyzes the Average Length of Stay (ALOS) across different distribution channels by integrating booking data with channel descriptions. Understanding which platforms bring guests who stay longer allows the hotel to prioritize marketing spend on channels that maximize inventory occupancy.

![Channel Stay Performance](../images/distribution_and_loyalty_analysis/channel_avg_length_of_stay.PNG)

**Business Analysis:** The data identifies **Agencies and Tour Operators** as the leaders in Average Length of Stay with **3.70 nights**, closely followed by **Direct Sales at 3.06 nights**. This indicates that leisure-oriented channels are key to reducing room turnover costs and maximizing occupancy per booking. In contrast, **Corporate Agreements** and **Global Distribution Systems (GDS)** show shorter averages (**2.25 and 1.70 nights** respectively), which aligns with the fast-paced nature of business travel. To optimize profitability, the hotel should design "extended stay" incentives specifically for these shorter-duration channels to increase their overall contribution to total room nights.

## 02 | Loyalty & Acquisition Channels: Where Do Repeat Guests Book?

**Business Question:** Do loyal guests (repeat customers) prefer to book directly with the hotel or do they continue to use intermediaries like travel agencies?

In [ ]:
WITH loyal_guest_channels AS (
	SELECT
		CASE
			WHEN hb.distribution_channel = 'Direct' THEN 'Direct'
			WHEN hb.distribution_channel IN ('TA/TO', 'GDS') THEN 'Agencies'
			ELSE 'Other'
		END AS channel_category
	FROM hotel_bookings hb
	WHERE 
		hb.is_repeated_guest = 1 
		AND hb.is_canceled = 0
		AND hb.distribution_channel <> 'Undefined'
)
SELECT
	lgc.channel_category,
	COUNT(*) AS total_bookings,
	ROUND(
		COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(),
		2
	) AS bookings_pct
FROM loyal_guest_channels lgc
GROUP BY lgc.channel_category
ORDER BY bookings_pct DESC;

### Visual Insight: Loyalty & Distribution Channel Preference

This visualization explores the booking behavior of returning guests (`is_repeated_guest`). By categorizing distribution channels into Direct, Agencies, and Others, the report identifies whether loyal customers are bypassing intermediaries to book through internal channels, which directly impacts the hotel's net profit margins.

![Repeated Guest Distribution](../images/distribution_and_loyalty_analysis/repeat_guest_channel_distribution.PNG)

**Business Analysis:** The data reveals a critical opportunity for margin optimization: only **27.21%** of repeated guests book directly with the hotel. A significant portion of loyal customers continues to use **Agencies (26.53%)** or **Other channels (46.26%)**, which implies that the hotel is paying commissions for guests who are already familiar with the brand. This dependency on third parties for returning visitors erodes net profit. To increase the "Direct Share," the hotel should implement a dedicated loyalty program that offers exclusive benefits—such as early check-in or better room rates—specifically for those who book through internal channels, effectively converting "Other" and "Agency" users into high-margin direct advocates.

## 03 | Operational Timeline: Weekly Occupancy Trends

**Business Question:** How does the hotel's actual occupancy (volume of non-canceled bookings) fluctuate week by week across different years?

In [ ]:
WITH weekly_occupancy_data AS (
	SELECT
		hb.arrival_year,
		hb.arrival_date_week_number AS week_num,
		hb.adr
	FROM hotel_bookings hb
	WHERE hb.is_canceled = 0
)
SELECT
	wod.arrival_year,
	wod.week_num,
	COUNT(*) AS total_occupancy
FROM weekly_occupancy_data wod
GROUP BY wod.arrival_year, wod.week_num
ORDER BY wod.arrival_year, wod.week_num;

### Visual Insight: Weekly Occupancy Timeline

This time-series visualization provides a granular view of hotel occupancy by aggregating non-canceled bookings on a weekly basis across different years. By synchronizing chronological data, the hotel can identify precise demand spikes, evaluate the impact of external events, and improve labor scheduling and inventory management.

![Weekly Occupancy Timeline](../images/distribution_and_loyalty_analysis/weekly_occupancy_timeline.PNG)

**Business Analysis:** The timeline reveals that while **Q2 and Q3** (weeks 18 to 35) maintain consistently high occupancy due to the summer season, a significant outlier appears in **week 44 of 2016**, reaching the highest historical peak of **752 bookings**. This October spike suggests a successful response to specific regional events or seasonal festivities like Halloween. Furthermore, the steady growth observed from 2015 to 2017 indicates an increasing brand presence. To optimize revenue during lower-occupancy periods (such as early Q1 and late Q4), the hotel should implement targeted promotional campaigns and dynamic pricing strategies, leveraging the "Halloween effect" discovered in this analysis to boost off-season demand.